LangChain 表达语言 (LCEL 


LangChain 表达式语言 (LCEL) 是一种声明式的链接 LangChain 组件的方法。LCEL 从第一天开始就被设计为支持将原型投入生产，无需更改代码，从最简单的“prompt + LLM”链到最复杂的链（我们已经看到人们在生产中成功运行了包含 100 多个步骤的 LCEL 链）。以下是您可能想要使用 LCEL 的几个原因：

### 一流的流式支持 
当您使用 LCEL 构建链时，您将获得最佳的第一个令牌时间（直到第一个输出块出现为止所经过的时间）。对于某些链，这意味着例如我们将令牌直接从 LLM 流式传输到流式输出解析器，然后您会以与 LLM 提供程序输出原始令牌相同的速率获得解析后的增量输出块。


### 异步支持 
使用 LCEL 构建的任何链都可以使用同步 API（例如在原型设计时在 Jupyter 笔记本中）以及异步 API（例如在LangServe服务器中）调用。这使得原型和生产中使用相同的代码成为可能，具有出色的性能，并且能够在同一服务器中处理许多并发请求。

### 优化并行执行 
每当您的 LCEL 链具有可以并行执行的步骤时（例如，如果您从多个检索器获取文档），我们都会在同步和异步接口中自动执行此操作，以尽可能减少延迟。

### 重试和回退 
为 LCEL 链的任何部分配置重试和回退。这是让您的链在规模上更可靠的好方法。我们目前正在努力添加对重试/回退的流式支持，这样您就可以获得额外的可靠性而无需任何延迟成本。

### 访问中间结果 
对于更复杂的链，在产生最终输出之前访问中间步骤的结果通常非常有用。这可用于让最终用户知道正在发生的事情，甚至只是调试您的链。您可以流式传输中间结果，并且它在每个LangServe服务器上都可用。

### 输入和输出模式 
输入和输出模式为每个 LCEL 链提供从链结构推断出的 Pydantic 和 JSONSchema 模式。这可用于验证输入和输出，是 LangServe 不可或缺的一部分。

### 无缝 LangSmith 
跟踪 随着您的链条变得越来越复杂，了解每一步究竟发生了什么变得越来越重要。使用 LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和可调试性。

###无缝 LangServe 部署
 使用 LCEL 创建的任何链都可以使用LangServe轻松部署。



## Runnable 接口
许多 LangChain 组件都实现了该Runnable协议，包括聊天模型、LLM、输出解析器、检索器、提示模板等

这是一个标准接口，可以轻松定义自定义链并以标准方式调用它们。标准接口包括：

stream：流回响应块
invoke：在输入上调用链
batch：在输入列表上调用链

它们还具有相应的异步方法，应与asyncio await语法一起使用以实现并发：

astream：异步流回响应块
ainvoke：在输入异步时调用链
abatch：异步调用输入列表上的链
astream_log：除了最终响应之外，还流回中间步骤的发生
astream_events：链中发生的betalangchain-core流事件（在0.1.14 中引入）


The input type and output type varies by component:

Component	Input Type	Output Type
Prompt	Dictionary	PromptValue
ChatModel	Single string, list of chat messages or a PromptValue	ChatMessage
LLM	Single string, list of chat messages or a PromptValue	String
OutputParser	The output of an LLM or ChatModel	Depends on the parser
Retriever	Single string	List of Documents
Tool	Single string or dictionary, depending on the tool	Depends on the tool

## 组件 Components

LangChain 为各种组件提供标准、可扩展的接口和外部集成，这些组件可用于使用 LLM 进行构建。有些组件是 LangChain 实现的，有些组件我们依赖第三方集成，还有一些则是混合的。

### Chat Models
语言模型使用一系列消息作为输入，并返回聊天消息作为输出（而不是使用纯文本）。这些传统上是较新的模型（较旧的模型通常是LLMs，见上文）。聊天模型支持为对话消息分配不同的角色，帮助区分来自人工智能、用户和系统消息等指令的消息。
尽管底层模型是消息输入、消息输出，但 LangChain 包装器还允许这些模型将字符串作为输入。这意味着您可以轻松地使用聊天模型代替 LLM。

当字符串作为输入传入时，它会转换为 HumanMessage 然后传递给底层模型。

LangChain 不提供任何聊天模型，而是依赖第三方集成。

在构建ChatModel时我们有一些标准化的参数：

model：模型的名称
ChatModels 还接受特定于该集成的其他参数。

## LLMs
语言模型以字符串作为输入并返回字符串。这些传统上是较旧的模型（较新的模型通常是ChatModels，见下文）。
尽管底层模型是字符串输入、字符串输出，但 LangChain 包装器还允许这些模型将消息作为输入。这使得它们可以与 ChatModel 互换。当消息作为输入传入时，它们将在后台格式化为字符串，然后再传递给底层模型。

LangChain 不提供任何 LLM，而是依赖第三方集成。一些语言模型将消息列表作为输入并返回一条消息。有几种不同类型的消息。所有消息都具有role、content和response_metadata属性。

## 提示模板
提示模板有助于将用户输入和参数转换为语言模型的指令。这可用于指导模型的响应，帮助其理解上下文并生成相关且连贯的基于语言的输出。

提示模板以字典作为输入，其中每个键代表提示模板中要填写的变量。

Prompt Templates 输出 PromptValue。此 PromptValue 可以传递给 LLM 或 ChatModel，也可以转换为字符串或消息列表。此 PromptValue 存在的原因是为了便于在字符串和消息之间切换。

有几种不同类型的提示模板

### 字符串
这些提示模板用于格式化单个字符串，通常用于较简单的输入。例如，构造和使用 PromptTemplate 的常见方法如下：

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

value = prompt_template.invoke({"topic": "cats"})
print(type(value))


<class 'langchain_core.prompt_values.StringPromptValue'>


In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant for {topic}"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic": "cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant for cats'), HumanMessage(content='Tell me a joke about cats')])

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='hi!')])

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part
])